<center>
    <h1><font size="16">Especialización en Estadística</font></h1>
    <h1><font size="12">SERIES DE TIEMPO</font></h1>
</center>

## 🎓 Taller Práctico – Modelos Avanzados para Series Temporales: Estado Espacial, Redes Neuronales y GAMs

### 📘 **Introducción**

En este taller exploraremos **modelos avanzados aplicados a series temporales**, poniendo énfasis en aquellos que permiten capturar patrones complejos como **tendencia, estacionalidad no lineal y dinámicas estructurales latentes**. A diferencia de enfoques clásicos como ARIMA, estos modelos ofrecen mayor flexibilidad para representar la evolución temporal de un fenómeno y generar pronósticos informados.

Nos centraremos en tres tipos de modelos:

1. **Modelos de Estado Espacial**: permiten descomponer la serie en componentes latentes (nivel, pendiente, estacionalidad) y realizar pronósticos robustos mediante el filtro de Kalman.
2. **Redes Neuronales**: ofrecen una aproximación no paramétrica para capturar relaciones no lineales a partir de rezagos temporales, incluyendo modelos autoregresivos y recurrentes.
3. **Modelos Aditivos Generalizados (GAMs)**: permiten modelar suavizadamente los efectos del tiempo y la estacionalidad mediante funciones spline, con alta interpretabilidad estructural.

> 🧠 Este taller no se centra en la construcción desde cero de los modelos, sino en **analizar, interpretar y comparar los resultados obtenidos**, incluyendo su descomposición, visualización y capacidad predictiva.

---

### 🎯 **Objetivos del Taller**

* Analizar e interpretar los componentes latentes estimados por modelos de estado espacial.
* Comparar la capacidad predictiva de distintos enfoques neuronales aplicados a una misma serie.
* Evaluar la descomposición y ajuste realizado por un modelo GAM en términos de tendencia y estacionalidad.
* Visualizar y comunicar de forma efectiva los resultados de cada modelo y su aplicabilidad en contextos reales.

---

### 🧰 **Requisitos Previos**

In [ ]:
library(KFAS)      # Modelos de estado espacial y filtro de Kalman con estructura flexible.
library(dlm)       # Alternativa para modelos de estado espacial con enfoque bayesiano/clásico.
library(mgcv)      # Modelos Aditivos Generalizados (GAMs) con funciones suavizadas tipo spline.

library(tsDyn)     # Modelos autoregresivos no lineales con redes neuronales (`nnetTs`).
library(nnet)      # Redes neuronales clásicas (perceptrones multicapa).
library(RSNNS)     # Implementación de redes neuronales recurrentes (Elman, Jordan) y avanzadas.
library(caret)     # Framework unificado para entrenamiento y validación de modelos (incluye redes neuronales).

library(fpp3)      # Framework moderno para análisis de series temporales con `tsibble`, `fable` y `ggplot2`.
library(forecast)  # Modelos clásicos de pronóstico y funciones auxiliares (`nnetar`, `forecast`, `accuracy`).
library(ggplot2)   # Visualización de series, componentes suavizados y pronósticos.
library(zoo)       # Manipulación de series temporales indexadas y conversión a `yearmon`.
library(lubridate) # Extracción y manejo de componentes temporales (año, mes).
library(tibble)    # Estructura moderna de data frames para flujos `tidy`.
library(dplyr)     # Manipulación eficiente y legible de datos (`mutate`, `filter`, `join`, etc.).

### 📥 1. Carga y transformación de datos

Trabajaremos con la serie mensual de pasajeros aéreos internacionales (`AirPassengers`), incluida por defecto en R. Esta serie contiene 144 observaciones mensuales desde enero de 1949 hasta diciembre de 1960.

In [ ]:
# Cargar la serie
data("AirPassengers")
ap <- AirPassengers

🔍 **Descripción**:

* La serie representa el número de pasajeros (en miles) que viajaron por avión cada mes.
* Se trata de una serie **no estacionaria**, tanto en media como en varianza.

---

#### 🔄 Transformación logarítmica

Aplicamos la transformación logarítmica para **estabilizar la varianza** y facilitar la interpretación en algunos modelos, como los de estado espacial, redes neuronales y GAMs.


In [ ]:
# Transformar a logaritmo natural
log_ap <- log(ap)

> 🧠 Esta transformación convierte los incrementos relativos en incrementos absolutos y suaviza la amplitud creciente de la estacionalidad.

### 🧱 **2. Definición del modelo de estado espacial estructural**

Iniciaremos con la construcción de un **modelo estructural clásico** que descompone la serie en dos componentes principales:

1. Una **tendencia local** con nivel y pendiente.
2. Una **estacionalidad mensual** representada mediante variables dummy.

Trabajamos sobre la serie en escala logarítmica, por lo que los efectos **multiplicativos se vuelven aditivos** en el modelo.

In [ ]:
# Modelo estructural: tendencia local + estacionalidad
modelo_ss <- SSModel(log_ap ~ 
                       SSMtrend(degree = 2, Q = list(matrix(NA), matrix(NA))) +
                       SSMseasonal(period = 12, sea.type = "dummy", Q = matrix(NA)),
                     H = matrix(NA))

#### 🧠 Explicación de los componentes

| Componente                        | Descripción                                                                                                           |
| :-------------------------------- | :-------------------------------------------------------------------------------------------------------------------- |
| `SSMtrend(degree = 2)`            | Modelo de **tendencia local lineal**, con dos estados: `nivel` y `pendiente`.                                         |
| `SSMseasonal(period = 12, dummy)` | Modelo de **estacionalidad mensual** con 11 variables dummy (meses).                                                  |
| `Q = list(...)`                   | Matriz de varianzas de los errores de estado (nivel, pendiente, estacionalidad). Se estiman por máxima verosimilitud. |
| `H = matrix(NA)`                  | Varianza del error de observación (ruido blanco). También estimada.                                                   |

---

> 📌 Este modelo es completamente **probabilístico y dinámico**, y permite analizar cómo evolucionan separadamente el nivel de la serie y la estacionalidad mes a mes.

### 🧮 **3. Estimación del modelo y aplicación del Filtro de Kalman**

Una vez definido el modelo estructural, el siguiente paso es **estimar los parámetros desconocidos** (las varianzas en $Q$ y $H$) mediante **máxima verosimilitud**, y luego aplicar el **Filtro de Kalman** para obtener los valores suavizados de los componentes latentes.

---

#### 🔢 Estimación de parámetros


In [ ]:
# Valores iniciales (en log-escala) para las varianzas:
# Orden: H (observación), Q1 (nivel), Q2 (pendiente), Q3 (estacionalidad)
iniciales <- log(c(0.01, 0.001, 0.001, 0.01))

# Estimación con método de optimización BFGS
fit <- fitSSM(modelo_ss, inits = iniciales, method = "BFGS")

# Extraer modelo ajustado con parámetros estimados
modelo_ajustado <- fit$model

# Visualizar resumen del modelo ajustado
modelo_ajustado


#### 📌 Explicación

* `fitSSM()` realiza la optimización de la verosimilitud para estimar los parámetros libres (valores `NA` en Q y H).
* Se usa el método **BFGS**, eficiente para problemas diferenciables.
* `modelo_ajustado` contiene ya los valores estimados y está listo para aplicar el **Filtro de Kalman**.

---

#### 🚦 Aplicar Filtro de Kalman + suavizado


In [ ]:
# Aplicar filtro de Kalman y suavizado de estados
kfs <- KFS(modelo_ajustado, smoothing = c("state", "signal"))

# Inspeccionar resumen del objeto
kfs

#### 🔍 ¿Qué contiene `kfs`?

* `kfs$alphahat`: valores suavizados de los **estados latentes** (nivel, pendiente, estacionalidad).
* `kfs$muhat`: serie suavizada reconstruida a partir de los componentes estimados.
* `kfs$V`: varianza de predicción.
* `kfs$v`: residuos de predicción.

Estos resultados se usarán en los pasos siguientes para **visualizar la tendencia, la estacionalidad y realizar pronósticos**.

### 💬 Preguntas

1. ¿Qué representa cada uno de los componentes del modelo de estado espacial (nivel, pendiente, estacionalidad)?
2. ¿Qué ventaja tiene representar la estacionalidad mediante variables dummy frente a usar funciones senoidales o splines?
3. ¿Qué nos dice el comportamiento del componente estacional sobre los ciclos anuales en la serie?
4. ¿Qué diferencia habría si usáramos una estacionalidad `bs = "cc"` con splines cíclicos como en GAMs?

---

### 📊 **4. Visualización de componentes suavizados**

Después de aplicar el filtro y suavizado de Kalman, es posible **descomponer la serie observada** en sus componentes latentes estimados: **nivel**, **pendiente** y **estacionalidad**.

En esta sección, visualizaremos cómo se comporta la **tendencia suavizada** en comparación con la serie original transformada.

---

#### 🔁 Preparar los datos para graficar


In [ ]:
# Crear un objeto ts con los componentes del modelo
ts_comp <- ts(cbind(
  Observado = log_ap,
  Tendencia = kfs$muhat,
  Nivel = kfs$alphahat[, "level"],
  Pendiente = kfs$alphahat[, "slope"],
  Estacionalidad = kfs$alphahat[, grepl("sea", colnames(kfs$alphahat))]
), start = start(log_ap), frequency = frequency(log_ap))


🔍 Aquí estamos construyendo una serie multivariada `ts` con:

* La serie observada (`log_ap`).
* La serie suavizada completa (`muhat`), que incluye todos los componentes.
* Los estados individuales: `nivel`, `pendiente`, y la **suma de efectos estacionales** (`sea_dummy1`, ..., `sea_dummy11`).

---

#### 📆 Construir un data frame con fechas


In [ ]:
library(zoo)
library(lubridate)
library(ggplot2)

# Extraer fechas a partir del índice de tiempo
fechas <- as.yearmon(time(ts_comp))

# Crear data.frame con observaciones y tendencia
df_comp <- data.frame(
  Fecha = as.Date(fechas),
  Observado = as.numeric(ts_comp[, "Observado"]),
  Tendencia = as.numeric(ts_comp[, "Tendencia"])
)


#### 📈 Graficar la tendencia suavizada


In [ ]:
ggplot(df_comp, aes(x = Fecha)) +
  geom_line(aes(y = Observado), color = "black", linewidth = 0.8) +
  geom_line(aes(y = Tendencia), color = "blue", linewidth = 0.8, linetype = "dashed") +
  labs(title = "Observado vs Tendencia (modelo de estado espacial)",
       y = "log(Pasajeros)", x = "Fecha") +
  theme_minimal()


### ✅ ¿Qué esperamos ver?

* La **línea negra** representa la serie log-transformada observada.
* La **línea azul discontinua** es la tendencia suavizada estimada por el modelo.
* Se espera que la tendencia siga el movimiento de fondo de la serie, **ignorando la estacionalidad** y el ruido aleatorio.

> 📌 Esta descomposición permite **interpretar el comportamiento subyacente** de la serie, separado de los efectos cíclicos y de corto plazo.

### 📊 **5. Visualización del componente estacional**

Una de las ventajas clave de los modelos de estado espacial es su capacidad para estimar la **componente estacional de forma separada y dinámica**, lo que permite analizar su comportamiento a lo largo del tiempo.

---

#### 🧮 Extracción de la estacionalidad


In [ ]:
# 1. Sumar los 11 efectos estacionales (uno por cada dummy)
estacionalidad_total <- rowSums(kfs$alphahat[, grepl("sea_dummy", colnames(kfs$alphahat))])

🔍 Cada fila de `kfs$alphahat` contiene los valores suavizados de los estados latentes. Al sumar los efectos `sea_dummy1` a `sea_dummy11`, se reconstruye la estacionalidad total en cada punto del tiempo.

---

#### 🧾 Construcción de la serie temporal


In [ ]:
# 2. Convertir a serie ts con la misma estructura que log_ap
ts_estacionalidad <- ts(estacionalidad_total, start = start(log_ap), frequency = frequency(log_ap))

#### 📆 Conversión a data.frame para graficar


In [ ]:
# 3. Crear data.frame con fechas y valores estacionales
df_est <- data.frame(
  Fecha = as.Date(as.yearmon(time(ts_estacionalidad))),
  Estacionalidad = as.numeric(ts_estacionalidad)
)

#### 📈 Graficar la estacionalidad suavizada


In [ ]:
# 4. Visualización
ggplot(df_est, aes(x = Fecha, y = Estacionalidad)) +
  geom_line(color = "darkgreen", linewidth = 0.8) +
  labs(title = "Componente Estacional Suavizado (Modelo de Estado Espacial)",
       x = "Fecha", y = "Estacionalidad") +
  theme_minimal()

### ✅ Interpretación esperada

* El gráfico muestra cómo varía la estacionalidad mes a mes a lo largo del tiempo.
* Se espera observar un patrón **cíclico repetitivo**, con meses consistentemente positivos (por ejemplo, junio–agosto) y otros negativos (enero, noviembre, etc.).
* La estacionalidad suavizada puede variar ligeramente entre años, permitiendo capturar **dinámicas estacionales no estrictamente constantes**.

> 🧠 Este tipo de representación es útil para **detectar cambios sutiles en la estructura estacional** a lo largo del tiempo.

### 📈 **6. Generación de pronósticos con modelo de estado espacial**

Una vez ajustado el modelo e interpretados los componentes latentes, es posible generar **pronósticos fuera de muestra** y sus respectivos **intervalos de predicción**. Esto se realiza fácilmente gracias a la estructura probabilística del modelo.

---

#### 🔮 Generar pronóstico a 24 meses



In [ ]:
# Predicción en escala logarítmica (con intervalo del 95%)
pred_kfas <- predict(modelo_ajustado, n.ahead = 24, interval = "prediction", level = 0.95)

# Transformar de log-escala a escala original (pasajeros)
pred_exp <- exp(pred_kfas)


🔍 `predict()` entrega un objeto con tres columnas:

* `"fit"`: valor pronosticado.
* `"lwr"` y `"upr"`: límites inferior y superior del intervalo de predicción.

---

#### 🧾 Visualización junto a la serie histórica



In [ ]:
# Visualizar pronóstico junto a la serie observada
forecast::autoplot(window(ap, end = c(1960, 12))) +
  autolayer(ts(pred_exp[, "fit"], start = c(1961, 1), frequency = 12), series = "Pronóstico") +
  autolayer(ts(pred_exp[, "lwr"], start = c(1961, 1), frequency = 12), series = "Límite inferior", linetype = "dashed") +
  autolayer(ts(pred_exp[, "upr"], start = c(1961, 1), frequency = 12), series = "Límite superior", linetype = "dashed") +
  labs(title = "Pronóstico con modelo de estado espacial",
       y = "Pasajeros", x = "Año") +
  theme_minimal()


#### 📤 Visualizar **solo el pronóstico**


In [ ]:
# Visualización solo del pronóstico y sus intervalos
forecast::autoplot(ts(pred_exp[, "fit"], start = c(1961, 1), frequency = 12), series = "Pronóstico") +
  autolayer(ts(pred_exp[, "lwr"], start = c(1961, 1), frequency = 12), series = "Límite inferior", linetype = "dashed") +
  autolayer(ts(pred_exp[, "upr"], start = c(1961, 1), frequency = 12), series = "Límite superior", linetype = "dashed") +
  labs(title = "Pronóstico con modelo de estado espacial",
       y = "Pasajeros", x = "Año") +
  theme_minimal()

### ✅ Interpretación esperada

* El modelo genera un **pronóstico suavizado** que **respeta la tendencia creciente y la estacionalidad** capturada.
* Las **bandas de predicción** reflejan la **incertidumbre creciente** a medida que se avanza en el horizonte.
* Este comportamiento es **típico de modelos estructurales** con múltiples componentes latentes.

## 🤖 Redes Neuronales aplicadas a Series Temporales

Las redes neuronales permiten modelar relaciones no lineales complejas entre los valores pasados de una serie y su comportamiento futuro. Aunque no están diseñadas específicamente para datos secuenciales como los modelos de estado espacial o ARIMA, pueden capturar patrones ocultos útiles para la predicción, especialmente si se utilizan ventanas de entrada (lags) adecuadas.

En esta sección trabajaremos con la serie `AirPassengers`, previamente transformada para estabilizar la varianza, y exploraremos diferentes enfoques neuronales para el pronóstico.

---

### 🔄 **1. Transformación logarítmica y visualización inicial**

Antes de aplicar cualquier red neuronal, transformamos la serie a logaritmo natural para facilitar la modelación y evitar predicciones negativas.


In [ ]:
# Transformación logarítmica
log_ap <- log(AirPassengers)

#### 📈 Visualización de la serie transformada

In [ ]:
library(forecast)
library(ggplot2)

autoplot(log_ap) +
  labs(title = "Serie log-transformada: AirPassengers",
       y = "log(Pasajeros)", x = "Año") +
  theme_minimal()


### 🧠 ¿Por qué usar el logaritmo?

* La transformación suaviza la **amplitud creciente de la estacionalidad**.
* Permite que modelos con salida lineal (como `nnet`) trabajen de manera más estable.
* Las predicciones en log-escala pueden transformarse de vuelta a escala original con `exp()`.

### ⚙️ **2. Modelo autoregresivo neuronal con `nnetar()`**

El paquete `forecast` incluye la función `nnetar()`, que ajusta de forma automática una **red neuronal autoregresiva no lineal (NNAR)**. Este modelo combina la flexibilidad de las redes neuronales con la estructura de los modelos AR tradicionales.

---

#### 🤖 Ajustar el modelo


In [ ]:
modelo_nnetar <- nnetar(log_ap)

* El modelo se entrena automáticamente seleccionando:

  * El número de retardos (lags) a usar.
  * La cantidad de neuronas en la capa oculta.
  * Si se incluye o no estacionalidad explícita.

---

#### 🔍 Ver resumen del modelo

In [ ]:
modelo_nnetar

> Este comando muestra la arquitectura utilizada (por ejemplo, NNAR(1,1,2)\[12]) y cuántas redes fueron promediadas para estabilizar el resultado.

---

#### 🔮 Generar pronóstico a 24 meses

In [ ]:
pron_nnetar <- forecast(modelo_nnetar, h = 24)

#### 📈 Visualizar el pronóstico

In [ ]:
autoplot(pron_nnetar) +
  labs(title = "Pronóstico con red neuronal autoregresiva (nnetar)",
       y = "log(Pasajeros)", x = "Año") +
  theme_minimal()

### ✅ Interpretación esperada

* El modelo genera una **serie suavizada de pronósticos** en log-escala, incluyendo bandas de predicción.
* Se espera que la red capture tanto la **tendencia global** como **patrones estacionales**, aunque puede suavizarlos en exceso si la arquitectura es muy simple.

> 📌 `nnetar()` es una excelente herramienta para establecer un baseline no lineal, sin necesidad de configurar manualmente la red.

### ⚙️ **3. Red neuronal personalizada con `nnet()`**

En esta etapa construiremos manualmente una red neuronal para pronóstico utilizando la función `nnet()` del paquete base. A diferencia de `nnetar()`, este enfoque nos permite **definir directamente la estructura de entrada y controlar el entrenamiento** paso a paso.

---

#### 🧮 Crear matriz de entrada (ventana de 12 lags)

In [ ]:
library(nnet)
library(tibble)

# Definir número de retardos
lags <- 12

# Construir matriz con lags + objetivo
x <- embed(log_ap, lags + 1)
y <- x[, 1]         # valor objetivo
X <- x[, -1]        # variables predictoras


> 📌 `embed()` crea una matriz en la que cada fila contiene los 12 valores anteriores (lagged inputs) y la respuesta.

---

#### ✂️ Separar conjunto de entrenamiento



In [ ]:
# Usamos todos los datos excepto los últimos 24 para entrenar
n <- nrow(X)
train_idx <- 1:(n - 24)

#### 🧠 Ajustar el modelo con `nnet()`

In [ ]:
# Entrenar red neuronal: 12 entradas, 5 neuronas ocultas, salida lineal
modelo_nnet <- nnet(X[train_idx, ], y[train_idx],
                    size = 5, linout = TRUE, trace = FALSE)

# Revisar estructura de la red
modelo_nnet

### ✅ ¿Qué hace esta red?

* **Arquitectura**: 12–5–1 (12 entradas, 5 nodos ocultos, 1 salida).
* **Activación**: funciones no lineales en capa oculta, salida lineal (`linout = TRUE`).
* Entrenada sobre datos escalados en log-escala, con 24 valores reservados para prueba.
* Sin validación cruzada automática, por lo que es **más sensible a sobreajuste**.

### 🔮 **4. Pronóstico recursivo con red neuronal personalizada (`nnet`)**

Una vez entrenado el modelo con una ventana deslizante de 12 retardos, generamos el pronóstico fuera de muestra **de manera recursiva**, es decir, alimentando el modelo con sus propias predicciones.

---

#### 🔁 Generar pronóstico a 24 pasos


In [ ]:
# Inicializar vector para almacenar predicciones
pred <- numeric(24)

# Tomar como entrada la última fila disponible del conjunto de entrenamiento
input <- X[n - 24, ]

# Pronóstico recursivo: una predicción alimenta la siguiente
for (i in 1:24) {
  pred[i] <- predict(modelo_nnet, matrix(input, nrow = 1))
  input <- c(pred[i], head(input, -1))  # actualizar entrada
}


> 📌 Este enfoque simula un escenario real de pronóstico, donde no se conocen los valores futuros verdaderos y el modelo debe apoyarse en sus propias salidas.

---

#### 🔄 Transformar y organizar resultados


In [ ]:
# Volver a escala original (pasajeros)
pred_exp <- exp(pred)

# Construir fechas de predicción
fechas_pred <- seq(as.Date("1961-01-01"), by = "month", length.out = 24)

# Data frame de predicción
df_pred <- data.frame(Fecha = fechas_pred, Pronostico = pred_exp)

# Serie histórica
df_hist <- data.frame(
  Fecha = as.Date(as.yearmon(time(AirPassengers))),
  Pasajeros = as.numeric(AirPassengers)
)


#### 📈 Visualización del pronóstico

In [ ]:
library(ggplot2)

ggplot() +
  geom_line(data = df_hist, aes(x = Fecha, y = Pasajeros), color = "black") +
  geom_line(data = df_pred, aes(x = Fecha, y = Pronostico), color = "blue") +
  labs(title = "Pronóstico con red neuronal simple (`nnet`)",
       y = "Pasajeros", x = "Fecha") +
  theme_minimal()

### ✅ ¿Qué observamos?

* La red logra continuar el patrón de crecimiento, aunque en general:

  * **Subestima la amplitud de la estacionalidad**.
  * Puede presentar **explosiones o colapsos** si no está bien regularizada.
* Este comportamiento es típico en redes **alimentadas recursivamente sin control de error acumulado**.

> ⚠️ Para mejorar este enfoque, se podrían usar técnicas de regularización, validación cruzada o redes con memoria como las recurrentes (`RSNNS`).

### 🧪 **5. Red neuronal con validación cruzada (`caret`)**

El paquete `caret` permite entrenar redes neuronales con control explícito de validación cruzada para datos temporales. En este caso, usaremos **12 retardos como predictores** y una estrategia de validación tipo *time slice*, adecuada para series temporales.

---

#### 🧮 Construcción del conjunto de datos


In [ ]:
library(caret)

# Crear matriz de datos con lags como predictores
data_nnet <- as.data.frame(embed(log_ap, 13))
colnames(data_nnet) <- c("y", paste0("lag", 1:12))

# Conjunto de entrenamiento (excluye los últimos 24 puntos)
train_nnet <- data_nnet[1:(nrow(data_nnet) - 24), ]

> 📌 Usamos 12 lags como variables independientes, y el valor actual como variable objetivo (`y`).

---

#### ⚙️ Definir estrategia de validación cruzada


In [ ]:
control <- trainControl(
  method = "timeslice",
  initialWindow = 96,
  horizon = 12,
  fixedWindow = TRUE
)

* Se utiliza un enfoque tipo **rolling forecasting origin**, que:

  * Mantiene una ventana de entrenamiento fija.
  * Evalúa el modelo en múltiples puntos del tiempo.
  * Es coherente con la estructura temporal de los datos.

---

#### 🧠 Entrenar la red neuronal


In [ ]:
set.seed(123)
modelo_caret <- train(
  y ~ ., data = train_nnet,
  method = "nnet",
  linout = TRUE,
  trace = FALSE,
  tuneLength = 5,
  trControl = control
)

modelo_caret

* `tuneLength = 5` explora automáticamente combinaciones de:

  * Número de neuronas ocultas (`size`).
  * Parámetro de regularización (`decay`).
* Se selecciona el modelo con menor RMSE en validación.

---

#### 🔮 Generar predicciones fuera de muestra


In [ ]:
# Últimos 24 registros (lagged inputs) como test
X_pred <- tail(data_nnet, 24)[, -1]

# Predicción en log-escala
pred_caret <- predict(modelo_caret, newdata = X_pred)

# Transformar a escala original
fechas_pred <- seq(as.Date("1961-01-01"), by = "month", length.out = 24)
df_pred_caret <- data.frame(
  Fecha = fechas_pred,
  Pronostico = exp(pred_caret)
)

#### 📈 Visualización del pronóstico


In [ ]:
ggplot() +
  geom_line(data = df_hist, aes(x = Fecha, y = Pasajeros), color = "black") +
  geom_line(data = df_pred_caret, aes(x = Fecha, y = Pronostico), color = "blue") +
  labs(title = "Pronóstico con red neuronal (`caret`)", y = "Pasajeros") +
  theme_minimal()

### ✅ Interpretación esperada

* El modelo se beneficia del proceso de validación cruzada, ajustando mejor la capacidad de generalización.
* Sin embargo, como depende solo de rezagos, **puede fallar al capturar estacionalidad compleja** o al mantener la tendencia con precisión en el largo plazo.
* La visualización permite detectar **desviaciones sistemáticas** y comparar con pronósticos anteriores (`nnet`, `nnetar`).

## 🔁 **6. Red neuronal recurrente con `RSNNS` (Elman)**

Las redes Elman son un tipo de **red neuronal recurrente (RNN)** que incorporan memoria corta mediante conexiones de retroalimentación. Son útiles para series temporales ya que pueden capturar **dependencias más profundas** que los modelos feedforward.

---

#### 🧠 Entrenar la red Elman


In [ ]:
library(RSNNS)

modelo_rsnns <- elman(
  X[train_idx, ], y[train_idx],
  size = c(5),
  learnFuncParams = c(0.1),
  maxit = 100
)

modelo_rsnns


#### 🔮 Pronóstico recursivo


In [ ]:
pred_rsnns <- numeric(24)
input <- X[n - 24, ]

for (i in 1:24) {
  pred_rsnns[i] <- predict(modelo_rsnns, matrix(input, nrow = 1))
  input <- c(pred_rsnns[i], head(input, -1))
}


#### 📊 Visualización


In [ ]:
df_pred_rsnns <- data.frame(
  Fecha = fechas_pred,
  Pronostico = exp(pred_rsnns)
)

ggplot() +
  geom_line(data = df_hist, aes(x = Fecha, y = Pasajeros), color = "black") +
  geom_line(data = df_pred_rsnns, aes(x = Fecha, y = Pronostico), color = "darkred") +
  labs(title = "Pronóstico con red neuronal recurrente (`RSNNS`)", y = "Pasajeros") +
  theme_minimal()


### ✅ Observaciones esperadas

* Las RNN pueden **capturar patrones de dependencia más largos**, pero:

  * Requieren más datos y entrenamiento más prolongado.
  * Son sensibles a la inicialización y número de iteraciones (`maxit`).
* La red puede producir **pronósticos planos** si no logra aprender la estructura estacional.

---

## 🔁 **7. Red autoregresiva con `tsDyn::nnetTs`**

`tsDyn` ofrece una forma rápida de ajustar redes autoregresivas con una interfaz especializada para series temporales.

---

#### 🧠 Ajustar la red

In [ ]:
library(tsDyn)

modelo_tsDyn <- nnetTs(log_ap[1:(length(log_ap) - 24)], m = 12, size = 5)
modelo_tsDyn


#### 🔮 Generar pronóstico a 24 pasos

In [ ]:
pred_tsDyn <- predict(modelo_tsDyn, n.ahead = 24)

df_pred_tsDyn <- data.frame(
  Fecha = fechas_pred,
  Pronostico = exp(pred_tsDyn)
)

#### 📈 Visualización

In [ ]:
ggplot() +
  geom_line(data = df_hist, aes(x = Fecha, y = Pasajeros), color = "black") +
  geom_line(data = df_pred_tsDyn, aes(x = Fecha, y = Pronostico), color = "purple") +
  labs(title = "Pronóstico con red neuronal autoregresiva (`tsDyn`)", y = "Pasajeros") +
  theme_minimal()

### ✅ Comentarios esperados

* `tsDyn` facilita el ajuste y predicción sin pasos manuales, ideal para exploración rápida.
* Puede generar **pronósticos razonables**, pero **no incluye validación cruzada ni regularización explícita**.
* A diferencia de `caret` o `nnetar`, **el modelo es sensible al sobreajuste y a la amplitud de los lags**.

### 💬 Preguntas

1.  ¿Qué ventajas ofrece el modelo de estado espacial frente a las redes neuronales cuando se busca descomponer una serie temporal en tendencia y estacionalidad? 
2.  ¿Por qué es importante aplicar la transformación logarítmica antes de ajustar redes neuronales a la serie `AirPassengers`? ¿Qué podría pasar si no se realiza? 
3.  En el modelo `nnetar()`, ¿cómo se interpreta la notación NNAR(1,1,2)\[12]? ¿Qué representa cada número? 
4.  ¿Cuál es la diferencia clave entre los pronósticos generados con `nnetar()` y los obtenidos con `nnet()` manual? ¿Qué riesgos introduce la predicción recursiva? 
5.  ¿Qué rol juega la validación cruzada tipo *rolling forecasting origin* utilizada con `caret`? ¿En qué se diferencia de una validación cruzada tradicional? 
6.  Al comparar los resultados de `RSNNS` y `tsDyn`, ¿por qué puede fallar una red neuronal en capturar adecuadamente la estacionalidad o la tendencia, incluso si tiene una arquitectura similar a otros modelos que sí lo logran? 

## 📐 Modelos Aditivos Generalizados (GAMs) con `mgcv`

Los modelos GAM permiten modelar series temporales de manera **flexible y estructurada**, separando la tendencia del tiempo y la estacionalidad mediante funciones suavizadas. Esta técnica es especialmente útil cuando no se desea imponer una estructura paramétrica rígida.

---

### ⚙️ **1. Preparación de datos**

Transformamos la serie `AirPassengers` en un `tibble` con variables explícitas para año, mes, tiempo y logaritmo:



In [ ]:
library(mgcv)
library(tibble)
library(dplyr)

df_ap <- tibble(
  Fecha = as.Date(as.yearmon(time(AirPassengers))),
  Año = as.numeric(format(Fecha, "%Y")),
  Mes = as.numeric(format(Fecha, "%m")),
  Tiempo = 1:length(AirPassengers),
  Pasajeros = as.numeric(AirPassengers),
  LogPasajeros = log(Pasajeros)
)

### 🧠 **2. Ajustar modelo GAM**

Usamos dos funciones suavizadas:

* `s(Tiempo)`: para capturar la **tendencia**.
* `s(Mes, bs = "cc")`: para modelar la **estacionalidad cíclica**.


In [ ]:
modelo_gam <- gam(LogPasajeros ~ s(Tiempo, k = 20) + s(Mes, bs = "cc", k = 12),
                  data = df_ap,
                  method = "REML")

### 📋 **3. Evaluar y visualizar el modelo**


In [ ]:
summary(modelo_gam)

par(mfrow = c(1, 2))
plot(modelo_gam, shade = TRUE, seWithMean = TRUE)


> Verifica la **significancia de los términos suavizados**, el **R² ajustado** y la **forma de los componentes suavizados** (tendencia y estacionalidad).

---

### 🧾 **4. Visualizar el ajuste**

In [ ]:
df_ap$Ajuste <- fitted(modelo_gam)

ggplot(df_ap, aes(x = Fecha)) +
  geom_line(aes(y = LogPasajeros), color = "black") +
  geom_line(aes(y = Ajuste), color = "blue") +
  labs(title = "GAM: Ajuste de log(Pasajeros)",
       y = "log(Pasajeros)", x = "Fecha") +
  theme_minimal()

### 🔮 **5. Generar predicción a 24 meses**

In [ ]:
# Crear nuevas fechas
ultimo_tiempo <- max(df_ap$Tiempo)
fechas_futuras <- seq(from = max(df_ap$Fecha) + 1, by = "month", length.out = 24)

# Armar data.frame de predicción
df_pred <- tibble(
  Fecha = fechas_futuras,
  Tiempo = (ultimo_tiempo + 1):(ultimo_tiempo + 24),
  Mes = as.numeric(format(fechas_futuras, "%m"))
)

# Predicción en log-escala con errores estándar
pred_log <- predict(modelo_gam, newdata = df_pred, se.fit = TRUE)

# Transformar a escala original
df_pred <- df_pred %>%
  mutate(
    LogPred = pred_log$fit,
    LogLI = LogPred - 1.96 * pred_log$se.fit,
    LogLS = LogPred + 1.96 * pred_log$se.fit,
    Pred = exp(LogPred),
    LI = exp(LogLI),
    LS = exp(LogLS)
  )

### 📈 **6. Visualizar predicción final**

In [ ]:
ggplot() +
  geom_line(data = df_ap, aes(x = Fecha, y = Pasajeros), color = "black") +
  geom_line(data = df_pred, aes(x = Fecha, y = Pred), color = "blue") +
  geom_ribbon(data = df_pred, aes(x = Fecha, ymin = LI, ymax = LS), alpha = 0.2, fill = "blue") +
  labs(title = "Predicción a 24 meses con GAM",
       x = "Fecha", y = "Pasajeros (escala original)") +
  theme_minimal()

### ✅ ¿Qué observamos?

* El GAM logra capturar tanto la **tendencia suave** como la **estacionalidad cíclica mensual**.
* Las bandas de predicción reflejan la **incertidumbre estimada** por el modelo.
* Es un modelo **explicativo e interpretable**, ideal para **análisis estructural y visualización**, aunque limitado en tareas de forecasting puro.

## 💬  Preguntas para discusión final del taller 

### 📌 Comparación y aplicación de modelos

1.  ¿Cuál de los tres enfoques modela de forma más transparente la estructura de una serie temporal? ¿Por qué? 
   *(Considera descomposición explícita, interpretabilidad y flexibilidad.)*
2.  ¿Qué tipo de modelo usarías si tuvieras una serie con cambios de régimen, intervenciones o datos faltantes? Justifica tu respuesta. 
3.  Si tu objetivo principal es la precisión del pronóstico en el corto plazo, ¿cuál modelo preferirías y por qué? 

---

### 🧠 Reflexión metodológica

4.  ¿Qué riesgos implica utilizar pronósticos recursivos con redes neuronales como `nnet()` o `RSNNS`? ¿Cómo podrían mitigarse? 
5.  ¿Cómo se diferencia el enfoque de estacionalidad en los GAMs respecto al modelo de estado espacial y las redes neuronales autoregresivas? 
6.  ¿Crees que combinar estos enfoques (por ejemplo, GAM + RNN o GAM + estado espacial) podría ser útil en algún contexto? ¿Qué ganarías o perderías con dicha combinación? 
